Instalando Pacotes

In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install pandas

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from concurrent.futures import ThreadPoolExecutor

Função + extraindo cidades

In [ ]:


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

# Função para extração de links
def extract_links(url):
    # fazendo o GET
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    empty_div = soup.find('div', class_='list__empty')
    # Se encontrar a classe list__empty é pra ele retornar None
    if empty_div:
        print(f"Não há informações de links para {url}")
        return None
    else:
        # se não é pra ele puxar as informações da lista
        primeira_ul = soup.find('ul', class_='list__list')

        if primeira_ul:
            links = [a['href'] for a in primeira_ul.find_all('a')]
            return links
        else:
            return []

# link do site
url = "https://listacep.com/"

# extração dos estados
estados_links = extract_links(url)

# extração das cidades
cidades_links = []
for estado_link in estados_links:
    cidades_links.extend(extract_links(estado_link))

# caso ele encontre 2x a mesma informação ou repita algo que está na lista de estados por algum erro
cidades_links = list(set(cidades_links)) 
cidades_links = [cidade for cidade in cidades_links if cidade not in estados_links]
cidades_links

Extraindo bairros

In [ ]:
# como estava demorando muito com a outra função decidi utilizar uma outra função:

from concurrent.futures import ThreadPoolExecutor
def extract_links2(url, max_attempts=5):
    for attempt in range(max_attempts):
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            empty_div = soup.find('div', class_='list__empty')

            if empty_div:
                print(f"Não há informações de links para {url}")
                return []
            else:
                primeira_ul = soup.find('ul', class_='list__list')

                if primeira_ul:
                    links = [a['href'] for a in primeira_ul.find_all('a')]
                    return links
                else:
                    return []
        except Exception as e:
            print(f"Tentativa {attempt + 1} de {max_attempts} - Erro ao processar {url}: {e}")
            if attempt < max_attempts - 1:
                print("Tentando novamente...")
            else:
                print("Número máximo de tentativas atingido. Desistindo.")
                return None


with ThreadPoolExecutor(max_workers=40) as executor:
    bairros = list(executor.map(extract_links2, cidades_links))

bairros_links = [bairro for sublist in bairros for bairro in sublist]
bairros_links = list(set(bairros_links))



Extraindo Ruas

In [ ]:
with ThreadPoolExecutor(max_workers=40) as executor:
    ruas = list(executor.map(extract_links2, bairros_links))

ruas_links = [rua for sublist in ruas for rua in sublist]
ruas_links = list(set(ruas_links))

In [ ]:
def extract_rua_data(rua, max_attempts=5):
    for attempt in range(max_attempts):
        try:
            response = requests.get(rua, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            busca = soup.find('div', class_="top__more-infos")

            if busca:
                texto = busca.text

                dados = {}
                for linha in texto.split("\n"):
                    if linha.strip():
                        chave, valor = linha.strip().split(":", 1)
                        dados[chave] = valor

                return dados
            else:
                return None
        except Exception as e:
            print(f"Tentativa {attempt + 1} de {max_attempts} - Erro ao processar {rua}: {e}")
            if attempt < max_attempts - 1:
                print("Tentando novamente...")
            else:
                print("Número máximo de tentativas atingido. Desistindo.")
                return None

rua_data_list = []

with ThreadPoolExecutor(max_workers=40) as executor:
    rua_data_list = list(executor.map(extract_rua_data, ruas_links))

rua_data_list = [data for data in rua_data_list if data]

df = pd.DataFrame(columns=["Logradouro", "CEP", "Bairro", "Cidade", "Estado"])
df = df.append(rua_data_list, ignore_index=True)

grupos_estados = df.groupby('Estado')

with pd.ExcelWriter('DadosCeps_por_estado.xlsx') as writer:
    for estado, dados_estado in grupos_estados:
        dados_estado.to_excel(writer, sheet_name=estado, index=False)